In [1]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()

True

In [2]:
# template = """Question: {question}
# 
# Answer: Let's think step by step."""
# 
# prompt = PromptTemplate(template=template, input_variables=["question"])

In [3]:
llm = OpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
# llm_chain = LLMChain(prompt=prompt, llm=llm)

In [4]:
# question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
# 
# llm_chain.run(question)

In [4]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

system_template = """Create an informative and comprehensive answer for a given question based solely on the given documents. You must only use information from the given documents.
Use an unbiased and journalistic tone. Do not repeat text.
Cite the documents using [Document name] notation.
If multiple documents contain the answer, cite those documents like ‘as stated in [Document name 1], [Document name 2], etc.’.
You must include citations in your answer.
If the documents do not contain the answer to the question, say that  ‘answering is not possible given the available information.’
{context}

"""
messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
prompt = ChatPromptTemplate.from_messages(messages)

In [5]:
from langchain.embeddings import OpenAIEmbeddings

faiss_local_path = '../data/embedded_dataset/faiss/openai_1000/faiss_idx'
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

In [6]:
from langchain import FAISS

db = FAISS.load_local(faiss_local_path, embeddings)

In [7]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

conversation_chain = ConversationalRetrievalChain.from_llm(
        llm,
        db.as_retriever(search_kwargs={"k": 3}),
        memory=ConversationBufferMemory(
        memory_key="chat_history", return_messages=True, output_key="answer"
    ),
        return_source_documents=True,
        combine_docs_chain_kwargs={
            "document_prompt": PromptTemplate(
                input_variables=["page_content", "Source"],
                template="Document name: {Source}\nContext:\n{page_content}",
            ),
            "prompt": prompt,
        },
    )

In [8]:
response = conversation_chain('What are the problems with the military standard IP specification?')

In [9]:
print(response['answer'])



Answer: According to [RFC 963], some problems with the Military Standard Internet Protocol (MIL-STD-1777) specification include failure to insert the last fragment of an incoming datagram, which would cause datagrams to be delivered to the upper-level protocol with some data missing, and passing an incorrect value of the data length for reassembled datagrams to the ULP, with unpredictable consequences. Additionally, as the specification is written, these errors are of little consequence, since the test for reassembly completion will always fail, with the result that reassembled datagrams would never be delivered at all [RFC 963].
